In [ ]:
import pandas as pd
from datetime import date, datetime
!pip install xlsxwriter
import xlsxwriter

# **Funcoes**

In [ ]:
def especialidade(num):
    especialidades = {1:"C.CIRURGICA",
                                2:"GO",
                                3:"C.MEDICA",
                                4:"Pacientes sob cuidados prolongados",
                                5:"Psiquiatria",
                                6:"Tisiologia",
                                7:"PEDIATRIA",
                                8:"Reabilitação",
                                9:"Psiquiatria em Hospital Dia"}
    try:
        return especialidades[num]
    except:
        return num

def tipoAlta(num):
    motivoAlta = {"ALTA HOSPITALAR":[11,12,14,15,16,18,19,61,62,63,64,23],
                        "ÓBITO":[41,42,43,67,65],
                        "TRANSFERENCIA":[31]}
    if num in motivoAlta["ALTA HOSPITALAR"]:
        return "ALTA HOSPITALAR"
    elif num in motivoAlta['ÓBITO']:
        return "ÓBITO"
    elif num in motivoAlta["TRANSFERENCIA"]:
        return "TRANSFERENCIA"
    else:
        return num

def caraterInt(num):
    carater = {1:"E",
                    2:"U",
                    5:"U",
                    6:"U"}
    return carater[num]


def converterData(data):
    try:
        dia, mes, ano = data.split("/")
    except AttributeError:
        dia = data.day
        mes = data.month
        ano = data.year
        return datetime(ano,mes,dia).date()
    try:
        return datetime(int(ano),int(mes),int(dia)).date()
    except AttributeError:
        return data


def criarDataframe(dicionario,colunas):
    df = pd.DataFrame()
    contador = 0
    qtdColunas = len(colunas)
    for coluna in colunas:
        temp = []
        for linha in dicionario.values():
            if len(linha) == qtdColunas:
                temp.append(linha[contador])
            elif len(linha) == 0:
                pass
            elif len(linha) < qtdColunas:
                temp.append(linha[0][contador])
        df[coluna] = temp
        contador = contador+1
    if len(colunas) == 4 and colunas[1] == 'Especialidade Fat':
        df = df.sort_values(["Especialidade Fat", "Prontuario"],ascending=[True,True])
    elif len(colunas) == 6 and colunas[2] == "Carater":
        df = df.sort_values(['Carater', 'Prontuario'],ascending = [True,True])
    else:
        df = df.sort_values(['Prontuario'],ascending=True)
    return(df)

# **Bate_Faturamento**

In [ ]:
#Abrindo as Planilhas Faturamento(pe) e Movimentacao(pm)

pf = pd.read_excel("/content/esta 08-02-24.xls",engine='xlrd')

pf = pf.rename(columns={"AIH_DT_SAI":"Saida",
                        "AIH_DT_INT":"Entrada",
                        "AIH_PRONT":"Prontuario",
                        "AIH_MOT_COB":"Tipo de Alta",
                        "AIH_PAC_NOME":"Nome",
                        "AIH_PAC_SEXO":"Sexo",
                        "AIH_CLINICA":"Clinica",
                        "AIH_CAR_INT":"Carater Int",
                        "AIH_NUM_AIH":"AIH"})

pm = pd.read_excel("/content/Planilha de Controle Censo Diário - "\
                   "Janeiro 2024.xls",sheet_name = "Janeiro 2024")

pm = pm.rename(columns={"DATA.1":"Saida",
                        "DATA INTERNAÇÃO":"Entrada",
                        "PRONTUARIO":"Prontuario",
                       "STATUS":"Tipo de Alta",
                        "Voltar":"Nome",
                        "SEXO":"Sexo",
                        "ESPECIALIDADE":"Clinica",
                        "U":"Carater Int" })

In [ ]:
#Fazendo alteracoes nas planilha para facilitar.
pf["Clinica"] = list(map(especialidade,pf["Clinica"]))
pf["Tipo de Alta"] = list(map(tipoAlta,pf["Tipo de Alta"]))
pf["Entrada"] = list(map(converterData,pf["Entrada"]))
pf["Saida"] = list(map(converterData,pf["Saida"]))
pf["Carater Int"] = list(map(caraterInt,pf["Carater Int"]))

pf = pf.loc[(pf['Tipo de Alta'] == "ÓBITO") |
            (pf['Tipo de Alta'] == "ALTA HOSPITALAR") |
            (pf['Tipo de Alta'] == "TRANSFERENCIA")]

pf = pf[pf.columns[[3,4,5,12,21,37,48,57,0]]]
pf = pf.sort_values(["Entrada"],ascending=True)

pm = pm.loc[pm['Saida'].notnull()]
pm = pm.loc[(pm['Tipo de Alta'] == "ÓBITO") |
            (pm['Tipo de Alta'] == "ALTA HOSPITALAR") |
            (pm['Tipo de Alta'] == "TRANSFERENCIA")]

pm = pm.sort_values(["Entrada"],ascending=True)
pm = pm[pm.columns[[1,2,15,7,14,3,4,9]]]
pm["AIH"] = 0
pm["Entrada"] = list(map(converterData,pm["Entrada"]))
pm["Saida"] = list(map(converterData,pm["Saida"]))



#Pegando somente as saidas de Setembro.
pf = pf.loc[(pf['Saida'] >= datetime(2024,1,1).date())]
pf = pf.loc[(pf['Saida'] <= datetime(2024,1,31).date())]
pm = pm.loc[(pm['Saida'] <= datetime(2024,1,31).date())]

# **Faturamento >> Movimenta**

In [ ]:
#Criando algumas lista e dicionarios que seram utilizados posteriormente.
prontUnicospf = []
prontRepetidospf = []
prontUnicospm = []
prontRepetidospm = []
clinica = {}
tipoAlta = {}
carater = {}
ausentePm = {}
ausentePf = {}
erro = []

#Retirando os prontuarios unicos e repetidos da planilha PF
for prontuario in pf['Prontuario']:
    if prontuario not in prontUnicospf:
        prontUnicospf.append(prontuario)
    else:
        prontRepetidospf.append(prontuario)

prontRepetidospf = list(set(prontRepetidospf))

for prontuario in prontRepetidospf:
    prontUnicospf.remove(prontuario)

#Retirando os prontuarios unicos e repetidos da planilha PM
for prontuario in pm['Prontuario']:
    if prontuario not in prontUnicospm:
        prontUnicospm.append(prontuario)
    else:
        prontRepetidospm.append(prontuario)

prontRepetidospm = list(set(prontRepetidospm))

for prontuario in prontRepetidospm:
    prontUnicospm.remove(prontuario)

for prontuario in prontUnicospf:
    if prontuario in prontUnicospm:
        prontUnicospm.remove(prontuario)
    elif prontuario in prontRepetidospm:
        prontRepetidospm.remove(prontuario)

for prontuario in prontRepetidospf:
    if prontuario in prontUnicospm:
        prontUnicospm.remove(prontuario)
    elif prontuario in prontRepetidospm:
        prontRepetidospm.remove(prontuario)

#Removendo protunarios que foram divisão
for prontuarios in prontRepetidospf:
    dadosPf = pf.loc[pf["Prontuario"] == prontuarios]
    dadosPm = pm.loc[pm["Prontuario"] == prontuarios]
    for linha in range(0,len(dadosPf)):
        try:
            if dadosPf.iloc[linha,2] == dadosPf.iloc[linha+1,1]:
                dadosPf.iloc[linha,2] = dadosPf.iloc[linha+1,2]
                indice = dadosPf.index[linha]
                pf.loc[indice] = dadosPf.iloc[linha]
                pf.drop(dadosPf.index[linha+1],inplace = True)
        except IndexError:
            if linha == len(dadosPf)-1:
                pass
            else:
                erro.append(prontuario)

In [ ]:
#Batendo as informacoes dos prontuarios unicos entre as planilhas,
for prontuario in prontUnicospf:
    dadosPm = pm.loc[pm["Prontuario"] == prontuario]
    dadosPf = pf.loc[pf["Prontuario"] == prontuario]

    if len(dadosPm) < 1:
        ausentePm[prontuario] = [prontuario,dadosPf.iloc[0,4],
                                 dadosPf.iloc[0,7],
                                 dadosPf.iloc[0,3],
                                 dadosPf.iloc[0,8]]

    elif len(dadosPm) == 1:
        if dadosPf.iloc[0,3] != dadosPm.iloc[0,3]:
            carater[prontuario] = [prontuario,dadosPf.iloc[0,7],
                                   dadosPf.iloc[0,3],
                                   dadosPf.iloc[0,8],
                                   dadosPm.iloc[0,7],
                                   dadosPm.iloc[0,3]]

        if dadosPf.iloc[0,4] != dadosPm.iloc[0,4]:
            tipoAlta[prontuario] = [prontuario,dadosPf.iloc[0,4],
                                    dadosPf.iloc[0,8],
                                    dadosPm.iloc[0,4]]

        if dadosPf.iloc[0,7] != dadosPm.iloc[0,7]:
            clinica[prontuario] = [prontuario,dadosPf.iloc[0,7],
                                   dadosPf.iloc[0,8],
                                   dadosPm.iloc[0,7]]

    elif len(dadosPm) > 1:
        temp = []
        for linha in range(0,len(dadosPm)):
            try:
                if dadosPf.iloc[linha,3] != dadosPm.iloc[linha,3]:
                    carater[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,3],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7],
                                           dadosPm.iloc[linha,3]]

                if dadosPf.iloc[linha,4] != dadosPm.iloc[linha,4]:
                    tipoAlta[prontuario] = [prontuario,dadosPf.iloc[linha,4],
                                            dadosPf.iloc[linha,4],
                                            dadosPm.iloc[linha,4]]

                if dadosPf.iloc[linha,7] != dadosPm.iloc[linha,7]:
                    clinica[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7]]
            except IndexError:
                if linha < len(dadosPm)-1:
                    temp.append([prontuario,dadosPm.iloc[linha,4],
                                 dadosPm.iloc[linha,7],
                                 dadosPm.iloc[linha,3]])

                elif linha == len(dadosPm)-1:
                    temp.append([prontuario,dadosPm.iloc[linha,4],
                                 dadosPm.iloc[linha,7],
                                 dadosPm.iloc[linha,3]])

                    ausentePf[prontuario] = temp

In [ ]:
for prontuario in prontRepetidospf:
    dadosPm = pm.loc[pm["Prontuario"] == prontuario]
    dadosPf = pf.loc[pf["Prontuario"] == prontuario]
    linhasPf, linhasPm = len(dadosPf), len(dadosPm)

    if linhasPf == linhasPm:
        tempCarater = []
        tempTipoalta = []
        tempClinica = []
        for linha in range(0,linhasPf):
            if dadosPf.iloc[linha,3] != dadosPm.iloc[linha,3]:
                tempCarater.append([prontuario,dadosPf.iloc[linha,7],
                                    dadosPf.iloc[linha,3],
                                    dadosPf.iloc[linha,8],
                                    dadosPm.iloc[linha,7],
                                    dadosPm.iloc[linha,3]])

            if dadosPf.iloc[linha,4] != dadosPm.iloc[linha,4]:
                tempTipoalta.append([prontuario,dadosPf.iloc[linha,4],
                                     dadosPf.iloc[linha,4],
                                     dadosPm.iloc[linha,4]])

            if dadosPf.iloc[linha,7] != dadosPm.iloc[linha,7]:
                tempClinica.append([prontuario,dadosPf.iloc[linha,7],
                                    dadosPf.iloc[linha,8],
                                    dadosPm.iloc[linha,7]])

        carater[prontuario] = tempCarater
        tipoAlta[prontuario] = tempTipoalta
        clinica[prontuario] = tempClinica

    elif linhasPf > linhasPm:
        temp = []
        tempCarater = []
        tempTipoalta = []
        tempClinica = []

        for linha in range(0,linhasPf):
            try:
                if dadosPf.iloc[linha,3] != dadosPm.iloc[linha,3]:
                    tempCarater.append([prontuario,dadosPf.iloc[linha,7],
                                        dadosPf.iloc[linha,3],
                                        dadosPf.iloc[linha,8],
                                        dadosPm.iloc[linha,7],
                                        dadosPm.iloc[linha,3]])

                if dadosPf.iloc[linha,4] != dadosPm.iloc[linha,4]:
                    tempTipoalta.append([prontuario,dadosPf.iloc[linha,4],
                                         dadosPf.iloc[linha,4],
                                         dadosPm.iloc[linha,4]])

                if dadosPf.iloc[linha,7] != dadosPm.iloc[linha,7]:
                    tempClinica.append([prontuario,dadosPf.iloc[linha,7],
                                        dadosPf.iloc[linha,8],
                                        dadosPm.iloc[linha,7]])

            except IndexError:
                temp.append([prontuario,dadosPf.iloc[linha,4],
                             dadosPf.iloc[linha,7],
                             dadosPf.iloc[linha,3],
                             dadosPf.iloc[0,8]])

                ausentePm[prontuario] = temp

        carater[prontuario] = tempCarater
        tipoAlta[prontuario] = tempTipoalta
        clinica[prontuario] = tempClinica

    elif linhasPm > linhasPf:
        temp = []
        for linha in range(0,linhasPm):
            try:
                if dadosPf.iloc[linha,3] != dadosPm.iloc[linha,3]:
                    carater[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,3],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7],
                                           dadosPm.iloc[linha,3]]

                if dadosPf.iloc[linha,4] != dadosPm.iloc[linha,4]:
                    tipoAlta[prontuario] = [prontuario,dadosPf.iloc[linha,4],
                                            dadosPf.iloc[linha,4],
                                            dadosPm.iloc[linha,4]]

                if dadosPf.iloc[linha,7] != dadosPm.iloc[linha,7]:
                    clinica[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7]]

            except IndexError:
                if linha < linha-1:
                    temp.append([prontuario,dadosPm.iloc[linha,4],
                                 dadosPm.iloc[linha,7],
                                 dadosPm.iloc[linha,3]])

                elif linha == linha-1:
                    temp.append([prontuario,dadosPm.iloc[linha,4],
                                 dadosPm.iloc[linha,7],
                                 dadosPm.iloc[linha,3]])

                    ausentePf[prontuario] = temp

# **Movimenta >> Faturamento**

In [ ]:
for prontuario in prontUnicospm:
    dadosPm = pm.loc[pm["Prontuario"] == prontuario]
    dadosPf = pf.loc[pf["Prontuario"] == prontuario]

    if len(dadosPf) < 1:
        ausentePf[prontuario] = [prontuario,dadosPm.iloc[0,4],
                                 dadosPm.iloc[0,7],
                                 dadosPm.iloc[0,3]]

    elif len(dadosPf) == 1:
        if dadosPf.iloc[0,3] != dadosPm.iloc[0,3]:
            carater[prontuario] = [prontuario,dadosPf.iloc[0,7],
                                   dadosPf.iloc[0,3],
                                   dadosPf.iloc[0,8],
                                   dadosPm.iloc[0,7],
                                   dadosPm.iloc[0,3]]

        if dadosPf.iloc[0,4] != dadosPm.iloc[0,4]:
            tipoAlta[prontuario] = [prontuario,dadosPf.iloc[0,4],
                                    dadosPf.iloc[0,8],
                                    dadosPm.iloc[0,4]]

        if dadosPf.iloc[0,7] != dadosPm.iloc[0,7]:
            clinica[prontuario] = [prontuario,dadosPf.iloc[0,7],
                                   dadosPf.iloc[0,8],
                                   dadosPm.iloc[0,7]]

    elif len(dadosPf) > 1:
        temp = []
        for linha in range(0,len(dadosPf)):
            try:
                if dadosPf.iloc[linha,3] != dadosPm.iloc[linha,3]:
                    carater[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,3],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7],
                                           dadosPm.iloc[linha,3]]

                if dadosPf.iloc[linha,4] != dadosPm.iloc[linha,4]:
                    tipoAlta[prontuario] = [prontuario,dadosPf.iloc[linha,4],
                                            dadosPf.iloc[linha,4],
                                            dadosPm.iloc[linha,4]]

                if dadosPf.iloc[linha,7] != dadosPm.iloc[linha,7]:
                    clinica[prontuario] = [prontuario,dadosPf.iloc[linha,7],
                                           dadosPf.iloc[linha,8],
                                           dadosPm.iloc[linha,7]]

            except IndexError:
                if linha < len(dadosPf)-1:
                    temp.append([prontuario,dadosPf.iloc[linha,4],
                                 dadosPf.iloc[linha,7],
                                 dadosPf.iloc[linha,3]])

                elif linha == len(dadosPf)-1:
                    temp.append([prontuario,dadosPf.iloc[linha,4],
                                 dadosPf.iloc[linha,7],
                                 dadosPf.iloc[linha,3],
                                 dadosPf.iloc[linha,8]])

                    ausentePm[prontuario] = temp

In [ ]:
dfClinica = criarDataframe(clinica,["Prontuario",
                                    "Especialidade Fat",
                                    "AIH",
                                    "Especialidade Estat"])

dfTipoalta = criarDataframe(tipoAlta,["Prontuario",
                                      "Alta Fat",
                                      "AIH",
                                      "Alta Estat"])

dfCarater = criarDataframe(carater,["Prontuario",
                                    "Clinica Fat",
                                    "Carater",
                                    "AIH",
                                    "Clinica Estat",
                                    "Carater Estat"])

dfSofat = criarDataframe(ausentePm,["Prontuario",
                                    "Alta Fat",
                                    "Clinica Fat",
                                    "Carater Fat",
                                    "AIH"])

dfSobenner = criarDataframe(ausentePf,["Prontuario",
                                       "Alta Fat",
                                       "Clinica Estat",
                                       "Carater Estat"])

In [ ]:
workbook = xlsxwriter.Workbook('Bate Faturamento.xlsx')
worksheet = workbook.add_worksheet('Clinicas')

cell_format1 = workbook.add_format()
cell_format1.set_font_size(12)
cell_format1.set_bold()
cell_format1.set_align('center')
cell_format1.set_border(1)
cell_format1.set_bg_color('#CFCFCF')

cell_format2 = workbook.add_format()
cell_format2.set_align('center')
cell_format2.set_border(1)


colunas = ['Prontuario','Faturamento', 'AIH_Internação','Estatistica']
tamanhoCol = [86.55, 100.15, 139, 94.11]

contador1 = 1
contador2 = 0
especialidade = ''


for coluna in colunas:
    worksheet.write(1,contador2,coluna,cell_format1)
    worksheet.set_column_pixels(contador2,contador2,tamanhoCol[contador2])
    contador2 += 1

for linha in range(0,len(dfClinica)):
    coluna = 0
    if dfClinica.iloc[linha][1] != dfClinica.iloc[linha-1][1]:
        contador1 += 1

    for elem in dfClinica.iloc[linha]:
        worksheet.write(contador1,coluna,elem,cell_format2)
        coluna += 1
    contador1 += 1

contador1 = 0
contador2 = 0

worksheet = workbook.add_worksheet('Pagina 2')


tamanhoCol = [86.55, 120.15, 139, 120.11,120]

for coluna in range(0,5):
    worksheet.set_column_pixels(contador2,contador2,tamanhoCol[contador2])
    contador2 += 1

if len(dfTipoalta) > 0:
    for linha in range(0,len(dfTipoalta)):
        coluna = 0
        if dfTipoalta.iloc[linha][1] != dfTipoalta.iloc[linha-1][1]:
            contador1 += 1
        for elem in dfTipoalta.iloc[linha]:
            worksheet.write(contador1, coluna, elem, cell_format2)
            coluna += 1
        contador1 += 1
    contador1 += 1

if len(dfCarater) > 0:
    for linha in range(0,len(dfCarater)):
        coluna = 0
        if (dfCarater.iloc[linha][2] != dfCarater.iloc[linha-1][2]):
            contador1 += 1
        for elem in dfCarater.iloc[linha]:
            worksheet.write(contador1, coluna, elem, cell_format2)
            coluna += 1
        contador1 += 1
    contador1 += 1

if len(dfSofat) > 0:
    for linha in range(0,len(dfSofat)):
        coluna = 0
        for elem in dfSofat.iloc[linha]:
            worksheet.write(contador1, coluna, elem, cell_format2)
            coluna += 1
            if coluna == len(dfSofat.iloc[linha]):
                worksheet.write(contador1,coluna,'Só Fat', cell_format2)
        contador1 += 1
    contador1 += 1

if len(dfSobenner) > 0:
    for linha in range(0,len(dfSobenner)):
        coluna = 0
        for elem in dfSobenner.iloc[linha]:
            worksheet.write(contador1, coluna, elem, cell_format2)
            coluna += 1
            if coluna == len(dfSobenner.iloc[linha]):
                worksheet.write(contador1, coluna, 'Só Benner', cell_format2)
        contador1 += 1
    contador1 += 1

workbook.close()